In [1]:
!pip install -U langchain-community
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00


In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
import warnings
warnings.filterwarnings('ignore')


In [ ]:
pdf = "/content/Resume_Ayush.pdf"
pdf_reader = PdfReader(pdf)
print(pdf_reader)

In [ ]:
# extrat text from each page separately
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

print(text)

Ayush Srivastava
Kanpur, Uttar Pradesh
/ne+91-7233846448 srivastavasasatyam@gmail.com Ayush Srivastava /gtbSriyush
EDUCATION
Vellore Institute of Technology 2021- Ongoing
B.Tech - Computer Science Engineering with spec in CPS -8.57 Chennai, Tamil Nadu, India
Aster Public School 2021
Intermediate - CBSE Board -87% Noida, Uttar Pradesh, India
Aster Public School 2019
Class 10 - CBSE -91% Noida, Uttar Pradesh, India
COURSEWORK / SKILLS
•Data Structures &
Algorithms
•Operating Systems•Database Management
System (DBMS)
•C++•Java
•HTML & CSS
•Reactjs•Nodejs
•Expressjs
•Flutter and Dart
PROJECTS
TodLearn |Flutter & Firebase
•Created a app using Flutter & Dart , with Firebase integration for authentication and database.
•The application is designed to provide accessible and engaging instruction in English and mathematics for
toddlers.
VideoFam |Flutter & Firebase
•Created a app using Flutter & Dart , with Firebase integration for authentication and database.
•The application is a straightforwa

In [ ]:
# Split the long text into small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700,
                                               chunk_overlap=200,
                                               length_function=len)

chunks = text_splitter.split_text(text=text)
chunks

['Ayush Srivastava\nKanpur, Uttar Pradesh\n/ne+91-7233846448 srivastavasasatyam@gmail.com Ayush Srivastava /gtbSriyush\nEDUCATION\nVellore Institute of Technology 2021- Ongoing\nB.Tech - Computer Science Engineering with spec in CPS -8.57 Chennai, Tamil Nadu, India\nAster Public School 2021\nIntermediate - CBSE Board -87% Noida, Uttar Pradesh, India\nAster Public School 2019\nClass 10 - CBSE -91% Noida, Uttar Pradesh, India\nCOURSEWORK / SKILLS\n•Data Structures &\nAlgorithms\n•Operating Systems•Database Management\nSystem (DBMS)\n•C++•Java\n•HTML & CSS\n•Reactjs•Nodejs\n•Expressjs\n•Flutter and Dart\nPROJECTS\nTodLearn |Flutter & Firebase',
 '•Data Structures &\nAlgorithms\n•Operating Systems•Database Management\nSystem (DBMS)\n•C++•Java\n•HTML & CSS\n•Reactjs•Nodejs\n•Expressjs\n•Flutter and Dart\nPROJECTS\nTodLearn |Flutter & Firebase\n•Created a app using Flutter & Dart , with Firebase integration for authentication and database.\n•The application is designed to provide accessible 

In [ ]:
chunks[0]

'Ayush Srivastava\nKanpur, Uttar Pradesh\n/ne+91-7233846448 srivastavasasatyam@gmail.com Ayush Srivastava /gtbSriyush\nEDUCATION\nVellore Institute of Technology 2021- Ongoing\nB.Tech - Computer Science Engineering with spec in CPS -8.57 Chennai, Tamil Nadu, India\nAster Public School 2021\nIntermediate - CBSE Board -87% Noida, Uttar Pradesh, India\nAster Public School 2019\nClass 10 - CBSE -91% Noida, Uttar Pradesh, India\nCOURSEWORK / SKILLS\n•Data Structures &\nAlgorithms\n•Operating Systems•Database Management\nSystem (DBMS)\n•C++•Java\n•HTML & CSS\n•Reactjs•Nodejs\n•Expressjs\n•Flutter and Dart\nPROJECTS\nTodLearn |Flutter & Firebase'

In [ ]:
chunks[1]

'•Data Structures &\nAlgorithms\n•Operating Systems•Database Management\nSystem (DBMS)\n•C++•Java\n•HTML & CSS\n•Reactjs•Nodejs\n•Expressjs\n•Flutter and Dart\nPROJECTS\nTodLearn |Flutter & Firebase\n•Created a app using Flutter & Dart , with Firebase integration for authentication and database.\n•The application is designed to provide accessible and engaging instruction in English and mathematics for\ntoddlers.\nVideoFam |Flutter & Firebase\n•Created a app using Flutter & Dart , with Firebase integration for authentication and database.\n•The application is a straightforward social media platform allowing users to upload, view, and comment on\nvideos.\nSOSH |MERN Full Stack Website'

"linkedin.com/in/gopiashokan \ngithub.com/gopiashokan \nWORK EXPERIENCE \nSenior Process Executive - Operations \nMahendra Next Wealth IT India Pvt Ltd \n05/2019 - 12/2022\n, \n \nNamakkal"

The above text is common(overlap) for both chunks[0] and chunks[1].
(chunk_overlap=200 - maximum length, it means length is not exceed 200)

In [3]:
!pip install transformers faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModel, pipeline
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
model = SentenceTransformer('all-MiniLM-L6-v2')
generator = pipeline("text-generation", model="microsoft/phi-2", device_map="auto")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
def huggingface_embedding_search(chunks, analyze):

    chunk_embeddings = model.encode(chunks)
    query_embedding = model.encode([analyze])

    index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
    index.add(np.array(chunk_embeddings))

    _, indices = index.search(np.array(query_embedding), k=3)
    return [chunks[i] for i in indices[0]]

def huggingface_role_suggestion(docs):

    prompt = f"Suggest suitable LinkedIn job roles based on this candidate's experience.\n\nJob Roles:"

    result = generator(prompt, max_new_tokens=100, num_return_sequences=1, do_sample=True)
    return result[0]['generated_text']



In [6]:
def huggingface_role_suggestion(docs):
    # Define the prompt to ensure the model suggests distinct job roles
    prompt = f"Suggest suitable LinkedIn job roles based on the experience provided in the resume. Please list distinct job roles and avoid repetition.\n\nJob Roles:"

    # Generate the result using Hugging Face model
    result = generator(prompt, max_new_tokens=100, num_return_sequences=1, do_sample=True)
    generated_text = result[0]['generated_text']

    # Assuming the roles are returned as a comma-separated string, we'll clean it up.
    roles = [role.strip() for role in generated_text.split(",")]

    # To ensure uniqueness, use a set to filter out duplicates
    seen_roles = set()
    unique_roles = []

    for role in roles:
        if role not in seen_roles:
            seen_roles.add(role)  # Mark the role as seen
            unique_roles.append(role)

    # Return the cleaned, unique job roles
    return ", ".join(unique_roles)


# # Example usage
# analyze = "What LinkedIn job roles should I apply to based on this resume?"

# # Replace `chunks` with the sections of the resume you're analyzing
# summary_result = huggingface_resume_role_suggestion(chunks, analyze)
# print(summary_result)

In [7]:
!pip install flask-ngrok pyngrok


In [8]:
!pip install flask-ngrok  # Ensure ngrok is installed
!ngrok authtoken $authtoken
!pip install flask-cors


  # Start ngrok manually to tunnel the Flask app on port 5000

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok  # Ensure pyngrok is installed

# Initialize Flask app
app = Flask(__name__)
CORS(app)
# Manually start ngrok
public_url = ngrok.connect(5000)
print(f"ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

@app.route("/upload", methods=["POST"])
def upload_resume():
    # Check if a file is included in the request
    if "resume" not in request.files:
        return jsonify({"error": "No resume file provided"}), 400

    resume = request.files["resume"]
    resume_path = "/content/" + resume.filename
    resume.save(resume_path)

    # Load and process the PDF
    pdf_reader = PdfReader(resume_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    # Split text into chunks for analysis
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200, length_function=len)
    chunks = text_splitter.split_text(text=text)

    # Define the query and run the job role suggestion function
    analyze_query = "What LinkedIn job roles should I apply to based on this resume?"
    job_roles = huggingface_role_suggestion(chunks)

    os.remove(resume_path)  # Clean up the file after processing

    # Send the job role suggestions as the response
    return jsonify({"result": job_roles})

# Start the Flask app
app.run()


ngrok tunnel "NgrokTunnel: "https://fac7-34-125-195-23.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
INFO:werkzeug:127.0.0.1 - - [17/Nov/2024 14:48:09] "POST /upload HTTP/1.1" 200 -


In [ ]:
!ngrok http 5000